In [31]:
ALPHA = 1

# Ridge Regression - some Theory

* Simalar to ordinary linear regression the starting point is the least squares cost function $\sum_{i=1}^N (y_i - \hat y_i)^2$ together with the linear ansatz  $y_i =  x_i^T \theta$.
* This gives the objective function
$$l = \sum_{i=1}^N (x_i^T \theta - \hat y_i)^2  + \alpha \lVert \theta \rVert_2 ^2,$$
where the additional regularization term $\alpha \lVert \theta \rVert_2 ^2$ with regularization strength $\alpha$ occurs.
* The optimum of this objective function can be found analytically
$$\theta ^* =  \left( X^T X  + \alpha \mathbb 1 \right)^{-1} X^T Y$$

* Note that we could have also taken the expected version (per datapoint) - i.e divide by the number of datapoints $N$ and achieve the same optimium as $N$ just acts as a constant.


### Todo: 
* Cholesky decomposition instead of inversion
* Connection to PCA (Hasies or Murphy)

# Get the data

In [32]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
Supervised = namedtuple("supervised", ["features", "target", "feature_names"])

def split_test_train(data):
    X_train, X_test, Y_train, Y_test = train_test_split(data.features, data.target, test_size = 0.2, random_state=5)
    train = Supervised(X_train, Y_train.reshape(-1, 1), feature_names=data.feature_names)
    test = Supervised(X_test, Y_test.reshape(-1, 1), feature_names=data.feature_names)
    return train, test 


def normalize(train: Supervised, test: Supervised):
    mu = train.features.mean(axis=0)
    std = train.features.std(axis=0)
    train_scaled = Supervised(features=(train.features - mu) / std, 
                              target=train.target, 
                              feature_names=train.feature_names) 
    test_scaled = Supervised(features=(test.features - mu) / std, 
                             target=test.target, 
                             feature_names=test.feature_names)
    return train_scaled, test_scaled


housing = fetch_california_housing()
train_data, test_data = normalize(*split_test_train(Supervised(housing.data, housing.target, housing.feature_names)))

# Ridge Regression via the Normal equation - with Tensorflow

In [33]:
def add_intercept(features):
    """Add intercept to features
    Todo: as an exercise use tensorflow"""
    m, n = features.shape
    return np.c_[np.ones((m, 1)), features]

    
def train(data, alpha):
    X = add_intercept(data.features)
    Y = data.target
    XT = tf.transpose(X)
    cov = tf.matmul(XT, X)
    one = tf.eye(cov.shape[0], dtype=tf.dtypes.float64)
    inv = tf.linalg.inv(cov + alpha * one)
    return tf.matmul(tf.matmul(inv, XT), Y)


def predict(data, theta):
    return tf.matmul( add_intercept(data.features),  theta)

theta_analytic = train(train_data, ALPHA)
y_pred_analytic  = predict(test_data, theta_analytic)

# Sklearn

In [34]:
# For comparison perform linear regression with scikit learn
from sklearn.linear_model import Ridge
#from sklearn.linear_model import LinearRegression

lin_model = Ridge(alpha=ALPHA, fit_intercept=True, normalize=False)
lin_model.fit(train_data.features, train_data.target)
theta_sklearn = tf.concat((lin_model.intercept_, lin_model.coef_.flatten()), axis=0)
y_pred_sklearn = lin_model.predict(test_data.features)

# Ridge Regression as a Neural Network with Tensorflow / Keras 
* Idea: use keras wit 1 layer, identitiy as activation function
* Assume that we could not find the minimizer analytically. 
* Thus we wish to find the minimzer computationally.
* Regularization is taken into account via bias and kernel regularizers in a dense layer.

In [35]:
from tensorflow.keras import regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


def plot_keras_training(history):
    plt.figure(figsize=(15,4))
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
    plotter.plot({'Basic': history}, metric = "mse")
    #plt.ylim([0, 3])
    #plt.xlim([0,30])
    plt.ylabel('MSE')
    plt.show()



ALPHA_TF = 0.5 * ALPHA / train_data.target.shape[0]
    
def build_model():
    """Regularization via kernel and bias regularizer"""
    model = keras.Sequential([layers.Dense(1 
                                           ,activation='linear'
                                           ,input_shape=[8]
                                           ,kernel_regularizer=regularizers.l2(ALPHA_TF) 
                                           ,bias_regularizer=regularizers.l2(ALPHA_TF))])
    optimizer = tf.keras.optimizers.Adam(0.001)
    #optimizer = tf.keras.optimizers.SGD(0.001)
    model.compile(loss="mse",
                  optimizer=optimizer,
                  metrics=["mse"])
    return model

model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [47]:
EPOCHS = 5000

history = model.fit(
  train_data.features, train_data.target,
  epochs=EPOCHS, 
  batch_size = train_data.features.shape[0], validation_split = 0.0, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

theta_keras = tf.concat((model.weights[1], tf.reshape(model.weights[0], [8])), axis=0)
y_pred_keras = model.predict(test_data.features)


Epoch: 0, loss:0.5223,  mse:0.5221,  
....................................................................................................
Epoch: 100, loss:0.5221,  mse:0.5219,  
....................................................................................................
Epoch: 200, loss:0.5219,  mse:0.5217,  
....................................................................................................
Epoch: 300, loss:0.5219,  mse:0.5217,  
....................................................................................................
Epoch: 400, loss:0.5218,  mse:0.5216,  
....................................................................................................
Epoch: 500, loss:0.5217,  mse:0.5215,  
....................................................................................................
Epoch: 600, loss:0.5217,  mse:0.5215,  
....................................................................................................
Epoch: 700, lo

### Got Stuck?
* The direct link for regularization is a bit tricky
* In fact the reparametrization of $\alpha$ seemed a bit odd. However, in the neural network above we optimized the MSE, 
$$l = \frac{1}{N} \sum_{i=1}^N (y_i - \hat y_i)^2$$
and the regularizer act as $2 \alpha \lVert w \rVert_2 ^2$ and  $2 \alpha \lVert b \rVert_2 ^2$ on weights an biases respectively. 
Putting things together we effectively optimize the quantity 
$$l = \frac{1}{N} \sum_{i=1}^N (y_i - \hat y_i)^2 + 2 \alpha ( \lVert w \rVert_2 ^2 +  \lVert b \rVert_2 ^2)$$
And therefore $\alpha \rightarrow \frac{\alpha}{2N}$ gives an equivalent objective function as in the ridge regression formulation above.
* It is illustrative to check if we start optimization at the known optimal weights (e.g. from sklearn) the optimizer should not move (up to some numerics)

In [59]:
# Get intial kernel and bias weights from sklearn solution
init_kernel = lambda shape, dtype: tf.constant(lin_model.coef_.reshape(shape), dtype=dtype)
init_bias = lambda shape, dtype: tf.constant(lin_model.intercept_.reshape(shape), dtype=dtype)

def build_model_at_optimum():
    """Ridge regression with weights initialized at optimum"""
    model = keras.Sequential([layers.Dense(1 
                                           ,activation='linear'
                                           ,input_shape=[8]
                                           ,kernel_regularizer=regularizers.l2(ALPHA_TF) 
                                           ,bias_regularizer=regularizers.l2(ALPHA_TF)
                                           ,kernel_initializer=init_kernel
                                           ,bias_initializer=init_bias
                                          )])
    optimizer = tf.keras.optimizers.Adam(0.001)
    model.compile(loss="mse",
                  optimizer=optimizer,
                  metrics=["mse"])
    return model


EPOCHS = 300
model_at_optimum = build_model_at_optimum()

history = model_at_optimum.fit(
  train_data.features, train_data.target,
  epochs=EPOCHS, 
  batch_size = train_data.features.shape[0], validation_split = 0.0, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, loss:0.5216,  mse:0.5214,  
....................................................................................................
Epoch: 100, loss:0.5216,  mse:0.5214,  
....................................................................................................
Epoch: 200, loss:0.5216,  mse:0.5214,  
....................................................................................................

# Ridge Regression using the estimator API
* Todo

# Comparision of results

In [52]:
def mse(y_true, y_pred):
    return tf.reduce_sum((y_true - y_pred)**2) / y_true.shape[0]

In [53]:
mse(test_data.target,  y_pred_sklearn) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.5363392915031693>

In [54]:
mse(test_data.target,  y_pred_analytic) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.5363429005540788>

In [55]:
mse(test_data.target,  y_pred_keras)

<tf.Tensor: shape=(), dtype=float64, numpy=0.536342870415293>

In [56]:
theta_sklearn

<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([ 2.06389635,  0.82785305,  0.11548134, -0.28098196,  0.32407719,
       -0.00341544, -0.04502633, -0.89539994, -0.86731315])>

In [57]:
tf.reshape(theta_analytic, (9,))

<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([ 2.06377137,  0.82785305,  0.11548134, -0.28098196,  0.32407719,
       -0.00341544, -0.04502633, -0.89539994, -0.86731315])>

In [58]:
theta_keras

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([ 2.0638301 ,  0.8278724 ,  0.11544083, -0.28107285,  0.324193  ,
       -0.00343003, -0.0450238 , -0.8957809 , -0.86769843], dtype=float32)>